In [ ]:
# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data

In [ ]:
# Data directory
DATADIR = '/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset'

In [ ]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [ ]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
# Import Keras
from keras import backend as K
sess = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()
print("X train shape :",X_train.shape)
print("X test shape :",X_test.shape)
print("Y train shape :",Y_train.shape)
print("Y train shape :",Y_test.shape)

AttributeError: ignored

In [ ]:
import pandas as pd
import numpy as np

def load_data():
    train_signals, train_labels = load_signals('train')
    test_signals, test_labels = load_signals('test')
    
    # Reshape the signal data into tensors
    train_signals = train_signals.reshape((train_signals.shape[0], train_signals.shape[1], 1))
    test_signals = test_signals.reshape((test_signals.shape[0], test_signals.shape[1], 1))
    
    # Convert the labels to one-hot encoding
    num_classes = np.max(train_labels) + 1
    train_labels = np.eye(num_classes)[train_labels]
    test_labels = np.eye(num_classes)[test_labels]
    
    # Convert the Pandas DataFrames to NumPy arrays
    X_train = train_signals.values
    X_test = test_signals.values
    Y_train = train_labels
    Y_test = test_labels
    
    return X_train, X_test, Y_train, Y_test


In [ ]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
import numpy as np
from pandas import read_csv
X_train = read_csv('/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset/train/X_train.txt', header=None, delim_whitespace=True)
X_test = read_csv('/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset/test/X_test.txt', header=None, delim_whitespace=True)
Y_train = read_csv('/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset/train/y_train.txt', header=None, delim_whitespace=True)
Y_test = read_csv('/content/drive/MyDrive/uci-har dataset/UCI HAR Dataset/test/y_test.txt', header=None, delim_whitespace=True)
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values
timesteps = X_train.shape[1]

In [ ]:
n_classes = _count_classes(Y_train)
print("X train shape: ", X_train.shape)
print("X test shape: ", X_test.shape)
print("Y train shape: ", Y_train.shape)
print("Y test shape: ", Y_test.shape)
print("Timesteps: ", timesteps)
print("Input dimensions: ", input_dim)
print("Number of classes: ", n_classes)

X train shape:  (7352, 561)
X test shape:  (2947, 561)
Y train shape:  (7352, 1)
Y test shape:  (2947, 1)
Timesteps:  561
Input dimensions:  561
Number of classes:  6


- Defining the Architecture of LSTM

In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                76032     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 76,230
Trainable params: 76,230
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30


ValueError: ignored

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [ ]:
score = model.evaluate(X_test, Y_test)

In [ ]:
score

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

### Assignment

#### 2 LSTM Layers

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,BatchNormalization,Conv2D,Flatten,TimeDistributed,Conv1D
from keras.regularizers import *
from keras.callbacks import LearningRateScheduler,TerminateOnNaN,EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.initializers import VarianceScaling
from keras.optimizers import *

In [ ]:
import math
def lr_decay(epoch):
    return float(0.0001 * math.pow(0.6, math.floor((1+epoch)/10)))
lr = LearningRateScheduler(lr_decay)
tm = TerminateOnNaN()
es = EarlyStopping(monitor = 'accuracy')
init = VarianceScaling(scale = 1.0,mode = 'fan_avg',distribution = 'normal')
adam = Adam(lr=0.001)
rmsprop = RMSprop(lr = 0.001)

In [ ]:

model = Sequential()
model.add(LSTM(32,activation = 'relu',return_sequences=True, input_shape=(timesteps, input_dim),recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003) ))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(LSTM(32,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(6, activation='sigmoid'))
model.summary()

In [ ]:
%%time
model.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
model.evaluate(X_test,Y_test)

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

### 3 LSTM Layers

In [ ]:
model = Sequential()
model.add(LSTM(16,activation = 'relu',return_sequences=True, input_shape=(timesteps, input_dim),recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003) ))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(LSTM(16,return_sequences=True,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(LSTM(16,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(6, activation='sigmoid'))
model.summary()

In [ ]:
%%time
model.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
model.evaluate(X_test,Y_test)

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [ ]:
model = Sequential()
model.add(LSTM(100,activation = 'relu',return_sequences=True, input_shape=(timesteps, input_dim),recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003) ))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(LSTM(100,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.0003)))
model.add(BatchNormalization())
model.add(Dropout(0.8))
# model.add(LSTM(28,return_sequences=True,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.0003)))
# model.add(BatchNormalization())
# model.add(Dropout(0.8))
# # model.add(LSTM(64,return_sequences=True,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.003)))
# # model.add(BatchNormalization())
# # model.add(Dropout(0.6))
# model.add(LSTM(16,recurrent_initializer="glorot_uniform",recurrent_regularizer=l2(0.0003)))
# model.add(BatchNormalization())
# model.add(Dropout(0.8))
model.add(Dense(n_classes, activation='softmax'))
model.summary()

In [ ]:
%%time
model.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),callbacks=[lr,tm],
          epochs=50,verbose = 1)

### Using CNN

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,BatchNormalization,Conv2D,Flatten,TimeDistributed,Conv1D,MaxPool1D
from keras.regularizers import *
from keras.callbacks import LearningRateScheduler,TerminateOnNaN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.initializers import VarianceScaling
from keras.optimizers import *

In [ ]:
import math
def lr_decay(epoch):
    return float(0.001 * math.pow(0.6, math.floor((1+epoch)/10)))
lr = LearningRateScheduler(lr_decay)
tm = TerminateOnNaN()
init = VarianceScaling(scale = 1.0,mode = 'fan_avg',distribution = 'normal')
adam = Adam(lr=0.001)
rmsprop = RMSprop(lr = 0.001)

#### Model 1 CNN

In [ ]:
model1 = Sequential()
model1.add(Conv1D(128,kernel_size=3,kernel_initializer='he_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.003),activation ='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.7))
model1.add(MaxPool1D(2))
# model1.add(Conv1D(64,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
# model1.add(BatchNormalization())
# model1.add(Dropout(0.5))
model1.add(Conv1D(32,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(MaxPool1D(2))
model1.add(Flatten())
model1.add(Dense(64,activation = 'relu'))
model1.add(Dropout(0.5))
model1.add(Dense(n_classes,activation = 'softmax'))
model1.summary()

In [ ]:
%%time
model1.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model1.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
model1.evaluate(X_test,Y_test)

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model1.predict(X_test)))

#### Model2 CNN

In [ ]:
model2 = Sequential()
model2.add(Conv1D(128,kernel_size=7,kernel_initializer='he_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.003),activation ='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.7))
model2.add(MaxPool1D(2))
# model2.add(Conv1D(64,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
# model2.add(BatchNormalization())
# model2.add(Dropout(0.5))
model2.add(Conv1D(64,kernel_size=7,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.7))
model2.add(MaxPool1D(2))
model2.add(Flatten())
model2.add(Dense(32,activation = 'relu'))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes,activation = 'softmax'))
model2.summary()

In [ ]:
%%time
model2.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model2.fit(X_train,
          Y_train,
          batch_size=64,
          validation_data=(X_test, Y_test),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
model2.evaluate(X_test,Y_test)

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model2.predict(X_test)))

#### Using Divide and Conquer Based Approach

In [ ]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))


In [ ]:
def load_y_bi(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    y[y<=3] = 0
    y[y>3] = 1
    return pd.get_dummies(y).as_matrix()

def load_data_bi():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train,y_test = load_y_bi('train'),load_y_bi("test")
    return X_train, X_test, y_train, y_test

In [ ]:
X_train_bi,X_test_bi,Y_train_bi,Y_test_bi  = load_data_bi()

In [ ]:
model_bi= Sequential()
model_bi.add(Conv1D(32,kernel_size=3,kernel_initializer='he_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.003),activation ='relu'))
#model_bi.add(BatchNormalization())
#model_bi.add(MaxPool1D(2))
model_bi.add(Conv1D(32,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
#model_bi.add(BatchNormalization())
model_bi.add(Dropout(0.6))
model_bi.add(MaxPool1D(2))
model_bi.add(Flatten())
model_bi.add(Dense(64,activation = 'relu'))
# model_bi.add(Dropout(0.5))
model_bi.add(Dense(2,activation = 'softmax'))
model_bi.summary()

In [ ]:
%%time
model_bi.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model_bi.fit(X_train_bi,
          Y_train_bi,
          batch_size=64,
          validation_data=(X_test_bi, Y_test_bi),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
model_bi.evaluate(X_test_bi,Y_test_bi)

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model_bi.predict(X_test_bi)))

In [ ]:
model_bi.save("model_bi_class.h5")

In [ ]:
def load_y_new(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    label_y = y>3
    new_y = y[label_y]
    return pd.get_dummies(new_y).as_matrix(),label_y

def load_data_new():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train_new, y_label_train = load_y_new('train')
    y_val_new,y_label_test = load_y_new('test')
    X_train_new = X_train[y_label_train]
    X_val_new = X_test[y_label_test]
    return X_train_new, X_val_new, y_train_new, y_val_new

In [ ]:
X_train_st,X_test_st,Y_train_st,Y_test_st  = load_data_new()

In [ ]:
model_st= Sequential()
model_st.add(Conv1D(30,kernel_size=3,kernel_initializer='glorot_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.0003),activation ='relu'))
model_st.add(BatchNormalization())
model_st.add(Dropout(0.7))
# model_st.add(MaxPool1D(2))
model_st.add(Conv1D(50,kernel_size=3,kernel_initializer='glorot_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
model_st.add(BatchNormalization())
model_st.add(Dropout(0.4))
model_st.add(Conv1D(100,kernel_size=3,kernel_initializer='glorot_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
model_st.add(BatchNormalization())
model_st.add(Dropout(0.4))
#model_st.add(MaxPool1D(2))
model_st.add(Flatten())
model_st.add(Dense(100,activation = 'relu'))
model_st.add(Dropout(0.6))
model_st.add(Dense(3,activation = 'softmax'))
model_st.summary()
# model_st = Sequential()
# model_st.add(Conv1D(100,kernel_size=3,kernel_initializer='he_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.0003),activation ='relu'))
# model_st.add(BatchNormalization())
# model_st.add(Dropout(0.7))
# # model_st.add(MaxPool1D(2))
# model_st.add(Conv1D(100,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
# model_st.add(BatchNormalization())
# model_st.add(Dropout(0.7))
# model_st.add(Conv1D(500,kernel_size=3,kernel_initializer='he_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
# model_st.add(BatchNormalization())
# model_st.add(Dropout(0.7))
# #model_st.add(MaxPool1D(2))
# model_st.add(Flatten())
# model_st.add(Dense(32,activation = 'relu'))
# model_st.add(Dropout(0.2))
# model_st.add(Dense(3,activation = 'softmax'))
# model_st.summary()

In [ ]:
%%time
model_st.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model_st.fit(X_train_st,
          Y_train_st,
          batch_size=64,
          validation_data=(X_test_st, Y_test_st),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
# Confusion Matrix
#print(confusion_matrix(np.argmax(Y_test_st,axis=1), np.argmax(model_st.predict(X_test_st),axis=1)))
model_st.evaluate(X_test_st,Y_test_st)

In [ ]:
print(confusion_matrix(np.argmax(Y_test_st,axis=1), np.argmax(model_st.predict(X_test_st),axis=1)))

In [ ]:
model_st.save("model_st_class.h5")

In [ ]:
def load_y_new(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    label_y = y<=3
    new_y = y[label_y]
    return pd.get_dummies(new_y).as_matrix(),label_y

def load_data_new():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train_new, y_label_train = load_y_new('train')
    y_val_new,y_label_test = load_y_new('test')
    X_train_new = X_train[y_label_train]
    X_val_new = X_test[y_label_test]
    return X_train_new, X_val_new, y_train_new, y_val_new

In [ ]:
X_train_dy,X_test_dy,Y_train_dy,Y_test_dy  = load_data_new()

In [ ]:
model_dy= Sequential()
model_dy.add(Conv1D(64,kernel_size=5,kernel_initializer='glorot_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.0003),activation ='relu'))
#model_dy.add(BatchNormalization())
#model_dy.add(MaxPool1D(2))
model_dy.add(Conv1D(32,kernel_size=5,kernel_initializer='glorot_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
#model_dy.add(BatchNormalization())
model_dy.add(Dropout(0.7))
# model_dy.add(Conv1D(32,kernel_size=7,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
# #model_dy.add(BatchNormalization())
# model_dy.add(Dropout(0.6))
model_dy.add(MaxPool1D(2))
model_dy.add(Flatten())
model_dy.add(Dense(16,activation = 'relu'))
model_dy.add(Dropout(0.2))
model_dy.add(Dense(3,activation = 'softmax'))
model_dy.summary()

In [ ]:
%%time
model_dy.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model_dy.fit(X_train_dy,
          Y_train_dy,
          batch_size=64,
          validation_data=(X_test_dy, Y_test_dy),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
# Confusion Matrix
#print(confusion_matrix(Y_test_dy, model_dy.predict(X_test_dy)))
model_dy.evaluate(X_test_dy,Y_test_dy)

In [ ]:
model_dy.save("model_dy_class.h5")

In [ ]:
model_dy.load_weights("model_dy_class.h5")

#### Sharpening Test Data

In [ ]:
def sharpen(x_test, sigma, alpha):
    d = x_test.shape[0]
    r = x_test.shape[1]
    c = x_test.shape[2]
    container = np.empty((d,r, c))
    i = 0

    for row in x_test:
        test = row
        blurred = ndimage.gaussian_filter(test, sigma)
        sharpened = test + alpha * (test - blurred)
        container[i] = sharpened
        i = i + 1
    return container

In [ ]:
from scipy import ndimage
alpha = np.arange(0.05, 2.55, 0.05)
sigma = np.arange(5, 10, 1)
from sklearn.metrics import accuracy_score,confusion_matrix
def display_output(X_test,Y_test):
    accuracy=[]
    for s in sigma:
        for a in alpha:
            # Sharpen test data with various sigma (for Gaussian filter) and alpha value combinations
            X_test_sharpen = sharpen(X_test, s, a)
            pred_dyna_sharpen = model_dy.predict(X_test_sharpen)
            accuracy.append(accuracy_score(Y_test, np.argmax(pred_dyna_sharpen, axis=1)))
    return(accuracy)
            #print(">>> sigma={}, alpha={:.2f}".format(s, a))
#             print(accuracy_score(np.argmax(Y_test,axis=1), np.argmax(pred_dyna_sharpen, axis=1)))
#             print(confusion_matrix(np.argmax(Y_test,axis=1), np.argmax(pred_dyna_sharpen, axis=1)))


In [ ]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
import warnings
warnings.filterwarnings("ignore")
X_train, X_test, Y_train, Y_test = load_data()
print("X train shape :",X_train.shape)
print("X test shape :",X_test.shape)
print("Y train shape :",Y_train.shape)
print("Y train shape :",Y_test.shape)

### Classification of Sitting and Laying

In [ ]:
X_train_st_sitlay = X_train_st[np.argmax(Y_train_st,axis=1)!=1]
X_test_st_sitlay = X_test_st[np.argmax(Y_test_st,axis=1)!=1]
Y_train_st_sitlay = Y_train_st[np.argmax(Y_train_st,axis=1)!=1]
Y_test_st_sitlay = Y_test_st[np.argmax(Y_test_st,axis=1)!=1]

In [ ]:
model_st_bi= Sequential()
model_st_bi.add(Conv1D(64,kernel_size=5,kernel_initializer='glorot_normal',input_shape=(timesteps, input_dim),kernel_regularizer = l2(0.0003),activation ='relu'))
#model_st_bi.add(BatchNormalization())
#model_st_bi.add(MaxPool1D(2))
model_st_bi.add(Conv1D(32,kernel_size=5,kernel_initializer='glorot_normal',kernel_regularizer = l2(0.0003),activation ='relu'))
#model_st_bi.add(BatchNormalization())
model_st_bi.add(Dropout(0.7))
# model_st_bi.add(Conv1D(32,kernel_size=7,kernel_initializer='he_normal',kernel_regularizer = l2(0.003),activation ='relu'))
# #model_st_bi.add(BatchNormalization())
# model_st_bi.add(Dropout(0.6))
model_st_bi.add(MaxPool1D(2))
model_st_bi.add(Flatten())
model_st_bi.add(Dense(16,activation = 'relu'))
model_st_bi.add(Dropout(0.3))
model_st_bi.add(Dense(3,activation = 'softmax'))
model_st_bi.summary()

In [ ]:
%%time
model_st_bi.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])

In [ ]:
%%time
# Training the model
result = model_st_bi.fit(X_train_st_sitlay,
          Y_train_st_sitlay,
          batch_size=64,
          validation_data=(X_test_st_sitlay, Y_test_st_sitlay),callbacks=[lr,tm],
          epochs=30,verbose = 1)

In [ ]:
print(confusion_matrix(np.argmax(Y_test_st_sitlay,axis=1), np.argmax(model_st_bi.predict(X_test_st_sitlay),axis=1)))
model_st_bi.evaluate(X_test_st_sitlay,Y_test_st_sitlay)

### Best Sigma and alpha for Test Sharpening

In [ ]:
accuracy=[]
alpha = np.arange(0.05, 2.55, 0.05)
sigma = np.arange(0, 5, 0.5) 
for s in sigma:
    for a in alpha:
        X_test_st_ = sharpen(X_test_st_,s,a)
        X_test_dy_ = sharpen(X_test_dy_,s,a)
        y_pred_st = model_st.predict_classes(X_test_st_)+3
        y_pred_dy = model_dy.predict_classes(X_test_dy_)
        total_y = np.concatenate([y_pred_st,y_pred_dy])
        accuracy.append(accuracy_score(np.argmax(y_test,axis=1),total_y))

### Stacking the models for Testing

In [ ]:
y_pred_bi = model_bi.predict_classes(X_test_bi)

In [ ]:
X_test_st_= X_test[y_pred_bi>0]   # Static class
X_test_dy_ = X_test[y_pred_bi<1]  # Dynamic Class

In [ ]:
Y_test_st_ = Y_test[y_pred_bi>0]
Y_test_dy_ = Y_test[y_pred_bi<1]

In [ ]:
y_test = np.concatenate([Y_test_st_,Y_test_dy_]) #

### Sharpening of Test Data

In [ ]:
X_test_st_ = sharpen(X_test_st_,5,0.05)
X_test_dy_ = sharpen(X_test_dy_,5,0.05)  

In [ ]:
y_pred_st = model_st.predict_classes(X_test_st_)+3
y_pred_dy = model_dy.predict_classes(X_test_dy_)

In [ ]:
total_y = np.concatenate([y_pred_st,y_pred_dy])

In [ ]:
print("Accuracy Using Divide and Conquer Method->",accuracy_score(np.argmax(y_test,axis=1),total_y))

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(np.argmax(y_test,axis=1),total_y),annot=True)

### Conclusion

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model","Accuracy","loss"]
x.add_row(["LSTM with one Layer",90.97,0.3087])
x.add_row(["LSTM with two Layer",91.78,0.3971])
x.add_row(["LSTM with three Layer",89.51,0.4863])
x.add_row(["Divide-Conquer Method",94.24,0.2042])
print(x)

1. All three model is getting confused between standing and sitting, after running so many code at last I got 91.8% accuracy in LSTM with 2 layers.
2. Some code gives nan as my losses and get stuck in same accuracy of 16.83% for many epochs .